In [4]:
import numpy as np
import open3d as o3d

# Robot Definition
The robot can be seen in the following image:

TODO: add image

It has 5 joints in a 2DoF shoulder, 1DoF elbow, 2DoF wrist configuration (the 2 wrist joints are both perpendicular to the elbow, while the elbow is parallel with the second shoulder joint).

In DH notation, the robot is defined as follows:

In [5]:
# Define robot geometry
#                 alpha_{i-1}  a_{i-1}  d_i  theta_i type  min  max
MOD_DH_PARAMS = [[0,           0,       0.1,   0,    'R', -150,  150],  # shoulder 1
                 [90,          0,       0,     90,   'R',  0,    180],  # shoulder 2
                 [0,           1,       0,     90,   'R',  0,    180],  # elbow
                 [90,          0,       1,     180,  'R',  30,   300],  # wrist 1
                 [90,          0,       0,     90,   'R',  0,    180]]  # wrist 2

REST_TRANSFORMS = [[]]

In [6]:
# Coordinate convention: x right, y into wall, z up
HAND_LENGTH = 0.2
FOREARM_LENGTH = 1
UPPERARM_LENGTH = 1
BASE_TO_SHOULDER = np.array([0, 0, 0.1])

In [14]:
# IK
def ik_elbow(x, z, elbow_in=True):
    # Uses law of cosines to find shoulder and elbow angle, given planar x/z
    a, b, c = UPPERARM_LENGTH, FOREARM_LENGTH, np.linalg.norm([x, z])
    gamma = np.arccos((a**2 + b**2 - c**2) / (2 * a * b))
    beta = np.arccos((a**2 + c**2 - b**2) / (2 * a * c))
    triangle_angle = np.arctan2(z, x)
    if elbow_in:
        shoulder_angle_from_horiz = triangle_angle + beta
        elbow_angle = -np.pi + gamma
    else:
        shoulder_angle_from_horiz = triangle_angle - beta
        elbow_angle = np.pi - gamma
    return np.rad2deg(shoulder_angle_from_horiz - np.pi/2), np.rad2deg(elbow_angle)

def ik_perpendicular(base_P_tip, elbow_in=True):  # position of brush tip relative to base frame
    angles = [0, 0, 0, 0, 0]
    base_P_wrist = base_P_tip - np.array([0, HAND_LENGTH, 0])
    shoulder_P_wrist_3d = base_P_wrist - BASE_TO_SHOULDER
    shoulder_P_wrist_2d = np.array([np.linalg.norm(shoulder_P_wrist_3d[:2]), shoulder_P_wrist_3d[2]])
    angles[1], angles[2] = ik_elbow(shoulder_P_wrist_2d[0], shoulder_P_wrist_2d[1], elbow_in=elbow_in)
    angles[0] = np.rad2deg(np.arctan2(shoulder_P_wrist_3d[1], shoulder_P_wrist_3d[0])) - 90
    angles[3] = 0  # TODO: this depends on base angle in a weird way
    angles[4] = 90 # TODO: this depends on base angle in a weird way
    return angles

In [18]:
# Unit test
base_P_tip = np.array([UPPERARM_LENGTH, HAND_LENGTH, FOREARM_LENGTH]) + BASE_TO_SHOULDER
expected_angles = [-90, -90, 90, -90, 90]
actual_angles = ik_perpendicular(base_P_tip, elbow_in=False)
np.testing.assert_allclose(actual_angles, expected_angles, err_msg='IK unit test failed')
expected_angles = [-90, 0, -90, -90, 90]
actual_angles = ik_perpendicular(base_P_tip, elbow_in=True)
np.testing.assert_allclose(actual_angles, expected_angles, err_msg='IK unit test failed')

1.0 1.0


AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0
IK unit test failed
Mismatched elements: 1 / 5 (20%)
Max absolute difference: 90.
Max relative difference: 1.
 x: array([-90., -90.,  90.,   0.,  90.])
 y: array([-90, -90,  90, -90,  90])